In [3]:
import pandas as pd
import requests
import json

In [5]:
df = pd.read_excel('Share of Voice - Grupos.xlsx', sheet_name='Sheet1', engine='openpyxl')

,cust_id,status cust_id,Classificação Atual,deals group on wpp?,link,Tipo Link,URL Corrigida
0,84848607,Confirmado,KAM - top 1000,Sim,https://chat.whatsapp.com/LBBRUqRFNeQElNvFBIvGiD,Link Quebrado,NaN
1,94733098,Confirmado,KAM - top 1000,Sim,https://chat.whatsapp.com/IR0gCURtqgZAJGne6meFbh,Grupo,NaN
2,224168003,Confirmado,KAM - top 1000,Sim,https://www.whatsapp.com/channel/0029VamngXq6x...,Misto,NaN
3,288063877,Confirmado,KAM - top 1000,Sim,https://chat.whatsapp.com/FYshNVla9hcBWbSoIwfGm3,Grupo,NaN
4,473343832,Confirmado,KAM - top 1000,Sim,https://chat.whatsapp.com/KvP3btvte3j6t7w7m9Q6h7,Grupo,NaN
...,...,...,...,...,...,...,...
424,2351879503,Pendente,top KAM Cross/Tech,Sim,NaN,NaN,NaN
425,1514602500,Pendente,top KAM Cross/Tech,Sim,NaN,NaN,NaN
426,2313173800,Pendente,top KAM Cross/Tech,Sim,NaN,NaN,NaN
427,547239300,Pendente,top KAM Cross/Tech,Sim,NaN,NaN,NaN
